In [1]:
from matplotlib.pyplot import draw, fill
import pyrender
import trimesh
from trimesh import remesh
import pandas as pd
import numpy as np
from trimesh import Trimesh
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
import pickle
import matplotlib.mlab as mlab
import seaborn as sns
import skeletor as sk
from view_data import view_mesh
import copy
from sklearn.preprocessing import MinMaxScaler
from project_statistics import get_outliers
from create_dataset import load_dataset,dir_to_sorted_file_list
from trimesh.repair import fill_holes
from trimesh.points import PointCloud
import ast
import random
from scipy.stats import wasserstein_distance
from random import choice

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('query_database.pkl', 'rb') as f:
    data = pickle.load(f)

In [14]:
with open('class_dict.pkl', 'rb') as f:
    class_dict = pickle.load(f)

class_dict

{'bee insect': 0,
 'spider arthropod': 1,
 'duck bird': 2,
 'dolphin underwater_creature': 3,
 'shark underwater_creature': 4,
 'feline quadruped': 5,
 'pig quadruped': 6,
 'skeleton body_part': 7,
 'human biped': 8,
 'human_arms_out human': 9,
 'trex biped': 10,
 'apatosaurus quadruped': 11,
 'brain body_part': 12,
 'face body_part': 13,
 'head body_part': 14,
 'torso body_part': 15,
 'castle building': 16,
 'dome_church building': 17,
 'roman_building building': 18,
 'two_story_home building': 19,
 'lighthouse building': 20,
 'multiple__peak_tent tent': 21,
 'bottle liquid_container': 22,
 'mug liquid_container': 23,
 'tank liquid_container': 24,
 'vase liquid_container': 25,
 'tv display_device': 26,
 'microchip 0': 27,
 'microscope 0': 28,
 'watch 0': 29,
 'desk_lamp lamp': 30,
 'acoustic_guitar guitar': 31,
 'piano musical_instrument': 32,
 'handgun gun': 33,
 'rifle gun': 34,
 'sword blade': 35,
 'butcher_knife blade': 36,
 'tire wheel': 37,
 'ice_cream 0': 38,
 'desk_with_hutch 

In [3]:
new_data = data.copy()
new_cls = pd.read_csv('class_labels_detailed.csv')

new_class_ids = []
for i in range(0,1814):
    new_class_id = list(new_cls[new_cls['id'] == i]['detailed_class_id'])
    if len(new_class_id)!=0:
        new_class_id = new_class_id[0]
        new_class_ids.append(new_class_id)
    else:
        new_class_ids.append(-1)

new_data['class'] = new_class_ids

new_data = new_data[new_data['class']!=-1]

In [21]:
new_data

,class,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature_1,curvature_2,...,D3_7,D3_8,D4_1,D4_2,D4_3,D4_4,D4_5,D4_6,D4_7,D4_8
5,0,-0.261642,0.422330,-0.119787,0.310935,1.0,0.127186,-0.026335,0.015866,0.018199,...,0.040183,0.070536,0.062792,0.259474,0.331735,0.219833,0.093546,0.026802,0.005236,0.000582
6,0,0.097291,1.233367,-0.132638,0.647950,1.0,0.558746,-0.026376,0.001970,0.008374,...,0.073101,0.211378,0.060142,0.262954,0.335814,0.227868,0.088447,0.021806,0.002829,0.000140
7,0,-0.363074,-0.185109,-0.131148,0.602948,1.0,-0.235055,-0.026488,0.000527,0.000000,...,0.035944,0.051549,0.079711,0.264226,0.296339,0.198696,0.109297,0.043025,0.008023,0.000682
8,0,-0.084140,0.431100,-0.119979,0.315248,1.0,-0.107428,-0.026382,0.016107,0.192617,...,0.067989,0.170555,0.066604,0.251375,0.320931,0.229509,0.101161,0.025381,0.004739,0.000301
16,1,-0.732833,-0.293262,-0.047704,-0.564564,1.0,-0.013457,-0.026477,0.033106,0.110878,...,0.009593,0.011761,0.137820,0.350961,0.282403,0.146951,0.060671,0.017541,0.003372,0.000281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,88,-0.096401,-0.593437,0.238536,-1.238260,1.0,-0.052897,-0.026490,0.133829,0.057001,...,0.007969,0.007367,0.280988,0.391352,0.254772,0.048899,0.019572,0.003633,0.000662,0.000120
1793,89,-0.635997,1.635528,-0.097206,-0.088215,1.0,-0.109831,-0.026475,0.003163,0.018450,...,0.041710,0.553367,0.194306,0.230620,0.163671,0.166700,0.130507,0.070500,0.029552,0.014144
1794,89,-0.628967,0.688594,-0.086371,-0.226206,1.0,-0.096914,-0.026411,0.022751,0.069841,...,0.033749,0.164413,0.295818,0.417031,0.201272,0.054958,0.020025,0.009250,0.001505,0.000140
1795,89,-0.752731,0.538449,-0.085358,-0.237839,1.0,0.026625,-0.026377,0.016546,0.000000,...,0.021062,0.279040,0.418410,0.328767,0.178485,0.030068,0.032054,0.009829,0.001284,0.001103


In [22]:
query = data.loc[221][1:]
query

surface_area     -0.588866
volume           -0.504440
compactness      -0.005640
sphericity       -0.784274
diameter          1.000000
rectangulairty   -0.403296
eccentricity     -0.026324
curvature_1       0.120042
curvature_2       0.026970
curvature_3       0.011105
curvature_4       0.019566
curvature_5       0.004231
curvature_6       0.008461
curvature_7       0.793760
curvature_8       0.015865
A3_1              0.294194
A3_2              0.184641
A3_3              0.139905
A3_4              0.118821
A3_5              0.081046
A3_6              0.073162
A3_7              0.071678
A3_8              0.036551
D1_1              0.128892
D1_2              0.067846
D1_3              0.084718
D1_4              0.135432
D1_5              0.195153
D1_6              0.134048
D1_7              0.099523
D1_8              0.154389
D2_1              0.101850
D2_2              0.173267
D2_3              0.166185
D2_4              0.176095
D2_5              0.131841
D2_6              0.100185
D

In [23]:
sampled_min_max = np.load('sampeld_min_max.npy')
sampled_mean_std = np.load('sampeld_mean_std.npy')

In [24]:
def standardize(data, idx):
    data_mean = sampled_mean_std[idx][0]  
    data_std = sampled_mean_std[idx][1]  
    normalized_descriptor_data = (data - data_mean)/data_std
    return normalized_descriptor_data

In [64]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(np.array(new_data.iloc[:,1:]), np.array(new_data['class']))

KNeighborsClassifier(n_neighbors=10)

In [65]:
new_data

,class,surface_area,volume,compactness,sphericity,diameter,rectangulairty,eccentricity,curvature_1,curvature_2,...,D3_7,D3_8,D4_1,D4_2,D4_3,D4_4,D4_5,D4_6,D4_7,D4_8
5,0,-0.261642,0.422330,-0.119787,0.310935,1.0,0.127186,-0.026335,0.015866,0.018199,...,0.040183,0.070536,0.062792,0.259474,0.331735,0.219833,0.093546,0.026802,0.005236,0.000582
6,0,0.097291,1.233367,-0.132638,0.647950,1.0,0.558746,-0.026376,0.001970,0.008374,...,0.073101,0.211378,0.060142,0.262954,0.335814,0.227868,0.088447,0.021806,0.002829,0.000140
7,0,-0.363074,-0.185109,-0.131148,0.602948,1.0,-0.235055,-0.026488,0.000527,0.000000,...,0.035944,0.051549,0.079711,0.264226,0.296339,0.198696,0.109297,0.043025,0.008023,0.000682
8,0,-0.084140,0.431100,-0.119979,0.315248,1.0,-0.107428,-0.026382,0.016107,0.192617,...,0.067989,0.170555,0.066604,0.251375,0.320931,0.229509,0.101161,0.025381,0.004739,0.000301
16,1,-0.732833,-0.293262,-0.047704,-0.564564,1.0,-0.013457,-0.026477,0.033106,0.110878,...,0.009593,0.011761,0.137820,0.350961,0.282403,0.146951,0.060671,0.017541,0.003372,0.000281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,88,-0.096401,-0.593437,0.238536,-1.238260,1.0,-0.052897,-0.026490,0.133829,0.057001,...,0.007969,0.007367,0.280988,0.391352,0.254772,0.048899,0.019572,0.003633,0.000662,0.000120
1793,89,-0.635997,1.635528,-0.097206,-0.088215,1.0,-0.109831,-0.026475,0.003163,0.018450,...,0.041710,0.553367,0.194306,0.230620,0.163671,0.166700,0.130507,0.070500,0.029552,0.014144
1794,89,-0.628967,0.688594,-0.086371,-0.226206,1.0,-0.096914,-0.026411,0.022751,0.069841,...,0.033749,0.164413,0.295818,0.417031,0.201272,0.054958,0.020025,0.009250,0.001505,0.000140
1795,89,-0.752731,0.538449,-0.085358,-0.237839,1.0,0.026625,-0.026377,0.016546,0.000000,...,0.021062,0.279040,0.418410,0.328767,0.178485,0.030068,0.032054,0.009829,0.001284,0.001103


In [91]:
# confusion matrix scaled for number of instances in a class
d = np.zeros((89,89))
class_acc = {}
K = 50
TP = {}
FP = {}
FN = {}
for cls in range(0,89):
    d[cls] = np.zeros(89)
    class_acc[cls] = 0
    TP[cls] = 0
    FP[cls] = 0
    FN[cls] = 0
    query_classes = np.array(new_data[new_data['class']==cls].iloc[:,1:])
    num_rows = 0
    for row in query_classes:
        num_rows +=1
        knn_query = np.array(row)[np.newaxis,...]
        knn_neighbors = knn.kneighbors(knn_query, K)[1][0]
    
        new_knn_neighbors = []
        for j in knn_neighbors:
            if j in new_data.index:
                new_knn_neighbors.append(j)
        
        pred_list = (map(lambda x: new_data.loc[x]["class"], new_knn_neighbors))
        pred_list = list(map(int, pred_list))
        
        target_list = np.repeat(cls, len(pred_list))
        #print(pred_list,target_list)
        #TP[cls] += pred_list.count(cls)
        #FP[cls] += K - pred_list.count(cls)
        
        d[cls] += [pred_list.count(j) for j in range(0,89)]

    class_acc[cls] = d[cls][cls]/np.sum(d[cls])
    TP[cls] = d[cls][cls]
    FP[cls] = np.sum(d[cls]) - TP[cls]

for i in range(0,89):
    FN[i] = np.sum(d[:,i]) - TP[i]

In [92]:
P = {}
R = {}
for i in range(0,19):
    P[i] = 0
    R[i] = 0
    P[i] = TP[i]/(TP[i] + FP[i])
    R[i] = TP[i]/(TP[i] + FN[i])

In [93]:
class_acc

{0: 0.023529411764705882,
 1: 0.011278195488721804,
 2: 0.0,
 3: 0.040983606557377046,
 4: 0.0,
 5: 0.03278688524590164,
 6: 0.0,
 7: 0.05405405405405406,
 8: 0.027827648114901255,
 9: 0.017889087656529516,
 10: 0.0,
 11: 0.0,
 12: 0.03571428571428571,
 13: 0.053864168618266976,
 14: 0.005934718100890208,
 15: 0.01098901098901099,
 16: 0.02976190476190476,
 17: 0.03761755485893417,
 18: 0.0903010033444816,
 19: 0.014598540145985401,
 20: 0.0,
 21: 0.0,
 22: 0.007142857142857143,
 23: 0.06711409395973154,
 24: 0.025423728813559324,
 25: 0.00411522633744856,
 26: 0.019801980198019802,
 27: 0.018404907975460124,
 28: 0.0,
 29: 0.0,
 30: 0.006369426751592357,
 31: 0.0,
 32: 0.0,
 33: 0.03508771929824561,
 34: 0.06372549019607843,
 35: 0.02452316076294278,
 36: 0.03333333333333333,
 37: 0.0,
 38: 0.0,
 39: 0.0056179775280898875,
 40: 0.011976047904191617,
 41: 0.027450980392156862,
 42: 0.0025188916876574307,
 43: 0.056074766355140186,
 44: 0.03464566929133858,
 45: 0.0,
 46: 0.0,
 47: 0.0,

In [94]:
global_acc = np.mean(list(class_acc.values()))
global_acc

0.010454565543188703

In [95]:
P

{0: 0.023529411764705882,
 1: 0.011278195488721804,
 2: 0.0,
 3: 0.040983606557377046,
 4: 0.0,
 5: 0.03278688524590164,
 6: 0.0,
 7: 0.05405405405405406,
 8: 0.027827648114901255,
 9: 0.017889087656529516,
 10: 0.0,
 11: 0.0,
 12: 0.03571428571428571,
 13: 0.053864168618266976,
 14: 0.005934718100890208,
 15: 0.01098901098901099,
 16: 0.02976190476190476,
 17: 0.03761755485893417,
 18: 0.0903010033444816}

In [241]:
global_p = np.mean(list(P.values()))
global_p

0.44748006379585326

In [240]:
R

{0: 0.45052631578947366,
 1: 0.5155172413793103,
 2: 0.4390681003584229,
 3: 0.4288079470198676,
 4: 0.4166666666666667,
 5: 0.42822966507177035,
 6: 0.5085106382978724,
 7: 0.4453125,
 8: 0.49894736842105264,
 9: 0.4406779661016949,
 10: 0.6266666666666667,
 11: 0.3983333333333333,
 12: 0.47237076648841353,
 13: 0.47607655502392343,
 14: 0.3544973544973545,
 15: 0.5333333333333333,
 16: 0.3938053097345133,
 17: 0.42962962962962964,
 18: 0.22972972972972974}

In [242]:
global_r = np.mean(list(R.values()))
global_r

0.446668794081212

In [234]:
TP

{0: 214.0,
 1: 299.0,
 2: 245.0,
 3: 259.0,
 4: 180.0,
 5: 179.0,
 6: 239.0,
 7: 171.0,
 8: 237.0,
 9: 208.0,
 10: 329.0,
 11: 239.0,
 12: 265.0,
 13: 199.0,
 14: 201.0,
 15: 320.0,
 16: 178.0,
 17: 174.0,
 18: 17.0}

In [235]:
FP

{0: 286.0,
 1: 201.0,
 2: 255.0,
 3: 241.0,
 4: 320.0,
 5: 321.0,
 6: 261.0,
 7: 324.0,
 8: 263.0,
 9: 292.0,
 10: 171.0,
 11: 261.0,
 12: 235.0,
 13: 301.0,
 14: 299.0,
 15: 180.0,
 16: 322.0,
 17: 326.0,
 18: 58.0}

In [236]:
FN

{0: 261.0,
 1: 281.0,
 2: 313.0,
 3: 345.0,
 4: 252.0,
 5: 239.0,
 6: 231.0,
 7: 213.0,
 8: 238.0,
 9: 264.0,
 10: 196.0,
 11: 361.0,
 12: 296.0,
 13: 219.0,
 14: 366.0,
 15: 280.0,
 16: 274.0,
 17: 231.0,
 18: 57.0}

In [ ]:
# metric for distance approach
# fine grained classes
# f1 and roc curve
# metrics for differnt query lengths
# presentation

In [161]:
global_acc = np.mean(list(class_acc.values()))
global_acc

0.44748006379585337

In [163]:
d

{0: array([214.,  20.,  38.,  25.,  11.,   8.,  11.,  14.,  10.,  10.,   9.,
         26.,  34.,  18.,  16.,   9.,  12.,  11.,   4.]),
 1: array([ 17., 299.,  51.,  12.,   5.,   4.,  20.,   2.,   4.,   2.,   2.,
          8.,   6.,  12.,  22.,  19.,  10.,   4.,   1.]),
 2: array([ 35.,  48., 245.,  38.,   3.,  10.,   2.,   2.,   4.,   2.,  13.,
         20.,  22.,  20.,  17.,   5.,  11.,   3.,   0.]),
 3: array([ 13.,  16.,  30., 259.,  10.,  24.,   5.,   9.,   6.,  20.,  31.,
          4.,   2.,  13.,  16.,   6.,  20.,  14.,   2.]),
 4: array([ 14.,  14.,   9.,  19., 180.,  33.,  14.,  19.,  15.,  16.,   3.,
         12.,   8.,  13.,  33.,  46.,  17.,  26.,   9.]),
 5: array([  8.,  13.,  12.,  44.,  37., 179.,   9.,  19.,  28.,  24.,  10.,
          6.,   0.,   5.,  23.,  32.,  18.,  27.,   6.]),
 6: array([ 20.,  30.,  16.,  14.,   5.,   9., 239.,  25.,   8.,   6.,   8.,
         18.,  16.,  12.,  38.,   5.,  19.,  11.,   1.]),
 7: array([ 14.,  15.,  11.,  23.,  23.,  19.,  41., 17